# Building Climate Transformers
Using PyTorch Forecasting package.

Recommend to use this note in a Colab notebook (GPU or TPU usage is required).

Copy [this Colab notebook](https://colab.research.google.com/drive/1P13Vjt9_swDSjXr83qXguIqkLZ-liDsN?usp=sharing) into your own Google Drive.

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer

import numpy as np
import pandas as pd

from pytorch_forecasting.metrics import QuantileLoss, RMSE

## Import data

In [ ]:
df = pd.read_csv("../../data/RoomClimate/Room-climate-ABC-clean.csv", parse_dates=['AbsTimestamp'])

df['Index'] = df.index

In [ ]:
df.head()

### Transform into a TimeSeries Dataset

In [ ]:
training_cutoff = "YYYY-MM-DD"  # day for cutoff

training = TimeSeriesDataSet(
        df, 
        target='Temperature',
        group_ids=['Room'],
        min_encoder_length=0,
        max_encoder_length=27,
        min_prediction_length=1,
        max_prediction_length=1,
        time_idx='Index',
        time_varying_unknown_categoricals=[],
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True
    )

## Create a validation and training dataloader

In [ ]:
validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)

train_dataloader = training.to_dataloader(train=True, batch_size=128, num_workers=2)
val_dataloader = validation.to_dataloader(train=True, batch_size=128, num_workers=2, shuffle=False)

## Create trainer

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=1, verbose=False, mode='min')

lr_logger = LearningRateMonitor()

trainer = pl.Trainer(
    max_epochs=100,
    gpus=0,
    gradient_clip_val=0.1,
    limit_train_batches=30,
    callbacks=[lr_logger, early_stop_callback]
)

In [ ]:
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=1,
    loss=RMSE(),
    log_interval=2,
    reduce_on_plateau_patience=4
)

In [ ]:
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

## Find optimal learning rate

In [ ]:
res = trainer.tuner.lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    early_stop_threshold=1000.0,
    max_lr=0.03
)

### Show optimal learning rate

In [ ]:
fig = res.plot(show=True, suggest=True)
fig.show()

In [ ]:
trainer.fit(
    tft, 
    train_dataloaders=train_dataloader, 
    val_dataloaders=val_dataloader
)